ACTUALIZACIÓN SERVICIO SOCIAL COMUNITARIO

In [ ]:
# Importación de las bibliotecas necesarias
import pandas as pd
import numpy as np
import time
from datetime import date
import mysql.connector
from mysql.connector import errorcode
import seaborn as sns


In [ ]:
try:
    # Establecer la conexión con la base de datos
    cnx = mysql.connector.connect(user='fese', password='F3$e.20-21',
                                  host='santandermoodle2022.cvxcmwgafwb2.us-east-2.rds.amazonaws.com',
                                  database='bitnami_moodle')
    cursor = cnx.cursor()
    
    # Definir la consulta SQL
    query = "SELECT mdl_user.id, username, firstname, lastname, email, city, description, \
            from_unixtime(timeaccess) as acceso FROM bitnami_moodle.mdl_user_lastaccess \
            left join mdl_user ON(mdl_user_lastaccess.userid=mdl_user.id) \
            where courseid=122;"
    
    # Ejecutar la consulta
    cursor.execute(query)
    
    # Convertir los resultados en un DataFrame de pandas
    result_ingresos = pd.DataFrame(cursor.fetchall())
    result_ingresos.columns = ['userId','username','firstname','lastname','email','city','description','lst_access']
    
    # Imprimir el número de usuarios que han ingresado al curso
    print("Usuarios que han ingresado al curso: {}".format(result_ingresos.shape))
    
    
    
except mysql.connector.Error as err:
    # Manejo de errores de la conexión a la base de datos
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Algo está mal con tu nombre de usuario o contraseña")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("La base de datos no existe")
    else:
        print(err)
finally:
    # Cerrar el cursor y la conexión a la base de datos
    if 'cursor' in locals() and cursor:
        cursor.close()
    if 'cnx' in locals() and cnx:
        cnx.close()
        
# Imprimir las primeras filas del DataFrame
result_ingresos.head()

In [ ]:
# Lectura de archivos fuente
file_update = "update_261224.xlsx"  # Nombre del archivo de actualización

# Leer el archivo consolidado desde un archivo CSV
consolidado = pd.read_csv("consolidado.csv")

# Leer el archivo de actualización desde un archivo Excel
# Asegúrate de que el archivo y la pestaña 'data' existen en la ruta especificada
# El parámetro header=0 indica que la primera fila (fila 0 en la indexación basada en cero) se debe usar como nombre de las columnas
update_file = pd.read_excel(f'actualizaciones/{file_update}', sheet_name='data', header=0)

# Eliminar los espacios sobrantes al principio y al final en todas las columnas
update_file = update_file.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Directorio de salida
path_gen = "output/"  # Ruta del directorio de salida

# Obtener la fecha de hoy en formato de cadena
date_today = str(date.today())

# Preparar los nombres de los archivos de salida
# Estos archivos se guardarán en la carpeta 'output' con la fecha de hoy en sus nombres
file_output_csv = f'{path_gen}mailchimpTest-{date_today}.csv'  # Archivo de salida para Mailchimp
file_upload_users_csv = f'{path_gen}altasTest-{date_today}.csv'  # Archivo de salida para las altas de usuarios
# Imprimir el número de estudiantes en el archivo consolidado
# La función shape[0] devuelve el número de filas en el DataFrame, que corresponde al número de estudiantes
print(f"Estudiantes en consolidado: {consolidado.shape[0]}")

# Imprimir el número de estudiantes en el archivo de actualización
print(f"Estudiantes en actualización: {update_file.shape[0]}")

# Identificar los nuevos registros en el archivo de actualización
# Esto se hace buscando los emails en el archivo de actualización que no están en el archivo consolidado
nuevos_registros = update_file[~update_file["email"].isin(consolidado["username"])]

# Imprimir el número de nuevos estudiantes por agregar
print(f"Nuevos estudiantes por agregar: {nuevos_registros.shape[0]}")
# Imprimir los nombres de las columnas en el DataFrame de actualización


In [ ]:
# Seleccionamos las columnas necesarias de los nuevos registros
df_columnas = nuevos_registros[["Terms Apply Request Date", "email","name", "lastName", "email",\
                                "university","phone", "otherUniversity", "Selecciona tu entidad federativa",\
                               "Ingresa tu número de matrícula de estudiante","sex","Nombre completo de tu carrera",\
                                "Periodo que cursa actualmente Semestre/Cuatrimestre/Trimestre ",\
                                "Selecciona tu perfil","¿Cómo te enteraste de esta convocatoria?","dniPassport",\
                               "Selecciona el área a la que se vinculan tus estudios actuales o más recientes"]]

# Renombramos las columnas para que coincidan con el formato de Moodle
df_columnas.columns = ["Date", "username","name","lastName","email","university","phone","otherUniversity","Entidad",\
                      "matricula","sexo","carrera","periodo","perfil","medio","dni","area_conocimiento"]

# Insertamos nuevas columnas con datos predefinidos
df_columnas.insert(2, "password", "SsC$2024", allow_duplicates=False)  # Contraseña predefinida
df_columnas.insert(6, "course1", "SSC2024", allow_duplicates=False)  # Curso predefinido
df_columnas.insert(11, "open_mail", "No", allow_duplicates=False)  # Configuración de correo electrónico predefinida

# Imprimimos el número de filas en el DataFrame de salida, que corresponde al número de estudiantes
print(f"Archivo de salida: {df_columnas.shape[0]}")


In [ ]:
# Creamos un diccionario vacío para almacenar las universidades
universidad = {}

# Recorremos el DataFrame fila por fila
#for i in range(len(df_columnas)):
    # Si la columna 'university' está vacía
#    if pd.isna(df_columnas.iloc[i]['university']):
#        # Asignamos el valor de 'otherUniversity' a la columna 'university'
#        df_columnas.at[df_columnas.index[i],'university'] = df_columnas.iloc[i]['otherUniversity']
# Si la columna 'university' está vacía, asignamos el valor de 'otherUniversity' a la columna 'university'
#df_columnas['university'] = df_columnas.apply(lambda row: row['otherUniversity'] if pd.isna(row['university']) else row['university'], axis=1)
df_columnas.loc[pd.isna(df_columnas['university']), 'university'] = df_columnas.loc[pd.isna(df_columnas['university']), 'otherUniversity']# Eliminamos la columna 'otherUniversity' ya que no es necesaria
del(df_columnas['otherUniversity'])

In [ ]:
# Crear un DataFrame con los usuarios de Mailchimp que han ingresado
df_usuarios_mailchimp_ingreso = consolidado[consolidado['email'].isin(result_ingresos['email'])].copy()

# Crear un DataFrame con los usuarios de Mailchimp que no han ingresado
df_usuarios_mailchimp_sin_ingreso = consolidado[~consolidado['email'].isin(result_ingresos['email'])].copy()

# Crear un DataFrame con los usuarios que han ingresado al sistema
df_usuarios_ingreso_sistema = result_ingresos.copy()

# Guardar el DataFrame de usuarios que han ingresado al sistema en un archivo CSV
df_usuarios_ingreso_sistema.to_csv("ingreso.csv", index = False)

# Imprimir el número de filas (usuarios) en cada DataFrame
print(f"consolidado: {consolidado.shape[0]}")
print(f"df_usuarios_mailchimp_ingreso: {df_usuarios_mailchimp_ingreso.shape[0]}")
print(f"df_usuarios_mailchimp_sin_ingreso: {df_usuarios_mailchimp_sin_ingreso.shape[0]}")
print(f"df_usuarios_ingreso_sistema: {df_usuarios_ingreso_sistema.shape[0]}")

# Añadir una nueva columna 'open_mail' al DataFrame de usuarios de Mailchimp que han ingresado, con todos los valores establecidos en "Sí"
df_usuarios_mailchimp_ingreso.loc[:, 'open_mail'] = "Sí"

# Concatenar los DataFrames 'consolidado' y 'df_columnas', eliminar los duplicados basándose en la columna 'email' y reiniciar el índice
final_dataframe = pd.concat([consolidado, df_columnas]).drop_duplicates(subset='email', keep='last').reset_index(drop=True)

# Concatenar los DataFrames 'df_columnas', 'df_usuarios_mailchimp_sin_ingreso' y 'df_usuarios_mailchimp_ingreso', eliminar los duplicados basándose en la columna 'email' y reiniciar el índice
df_mailchimp = pd.concat([df_columnas, df_usuarios_mailchimp_sin_ingreso, df_usuarios_mailchimp_ingreso]).drop_duplicates(subset='email', keep='last').reset_index(drop=True)

# Imprimir el número de filas y columnas en cada DataFrame
print(f"Shape of final_dataframe: {final_dataframe.shape}")
print(f"Shape of df_mailchimp: {df_mailchimp.shape}")

# Eliminar columnas innecesarias
df_mailchimp.drop(['Date', 'sexo', 'carrera', 'periodo', 'perfil', 'medio', 'dni', 'matricula', 'area_conocimiento'], axis=1, inplace=True)

# Renombrar columnas
df_mailchimp.columns = ["email","password","firstname","lastName","usuario","curso","IES","phone","Entidad","Correoabierto"]

In [ ]:
# Generación del archivo consolidado
final_dataframe.to_csv('consolidado.csv', index=False)

# Generación del archivo Mailchimp
df_mailchimp.to_csv(file_output_csv, index=False)
print("Archivo Mailchimp generado", file_output_csv)

# Crear una copia del DataFrame 'df_columnas'
df_columnas_copy = df_columnas.copy()

# Borrado de columnas
df_columnas_copy.drop(['Date', 'open_mail'], axis=1, inplace=True)

# Renombrar columnas
df_columnas_copy.columns = ['username', 'password', 'firstname', 'lastName', 'email', 'course1',\
                       'description','phone1','city','skype','yahoo','address','icq','msn',\
                       'url','idnumber','department']

# Escritura_archivo_salida
df_columnas_copy.to_csv(file_upload_users_csv, index=False)
print("Archivo Update  generado", file_output_csv)

# Imprimir la forma del DataFrame
print(df_columnas_copy.shape)